In [1]:
import os,sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
print(sys.executable)

c:\Users\2401-00010\.conda\envs\pycaretTest\python.exe


In [3]:
df = pd.read_csv('D:/bmsAnomalyDetection/application/Anomaly-detection/notebook/data/labelled_data.csv')
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,timestamps,BMS_state,BMS_soc,BMS_soh,BMS_bus_voltage,BMS_bus_current,BMS_isolation,BMS_max_cell_temp,BMS_max_cell_temp_id,BMS_min_cell_temp,BMS_min_cell_temp_id,BMS_max_cell_voltage,BMS_max_cell_voltage_id,BMS_min_cell_voltage,BMS_min_cell_voltage_id,LV_soc,LV_soh,LV_voltage,LV_current,LV_temperature,MCU_motor_speed,MCU_motor_avg_temp,OBC_mux,OBC_port_status,OBC_overvoltage_fault,OBC_overcurrent_fault,OBC_output_voltage,OBC_output_current,OBC_port_weld_fault,OBC_internal_voltage,OBC_internal_current,Anomaly,Anomaly_Score
0,0,2024-07-23 10:54:34.076000+05:30,3.0,95.000000,96.500000,443.800000,223.300000,1190.0,29.300000,1.0,27.300000,9.0,4.060000,7.0,3.920000,5.0,82.700000,96.200000,10.000000,0.900000,73.0,4301.600000,37.700000,0.0,0.0,0.0,0.0,348.000000,212.000000,0.0,345.000000,143.000000,0,301.415981
1,1,2024-07-23 10:54:39.076000+05:30,3.0,95.000000,96.106415,443.996792,222.414435,1190.0,29.102313,1.0,26.855204,9.0,4.050116,7.0,3.885405,5.0,85.220510,96.051735,11.482653,0.751735,73.0,4154.866780,38.688435,0.0,0.0,0.0,0.0,345.059976,251.200314,0.0,343.009653,161.410708,0,-47.702002
2,2,2024-07-23 10:54:44.076000+05:30,3.0,95.000000,95.712831,444.193585,221.528869,1190.0,28.904626,1.0,26.410408,9.0,4.040231,7.0,3.850810,5.0,87.741020,95.903469,12.965306,0.603469,73.0,4008.133560,39.676871,0.0,0.0,0.0,0.0,342.119953,290.400627,0.0,341.019306,179.821416,0,30.631203
3,3,2024-07-23 10:54:49.076000+05:30,3.0,95.094741,96.173705,445.763226,220.126256,1205.0,28.661967,2.0,26.304787,6.0,4.078085,8.0,3.850000,5.0,88.038033,95.519148,13.000000,0.600000,62.0,4073.967475,38.747870,0.0,0.0,0.0,0.0,344.309767,291.076093,0.0,339.028960,198.232124,0,5.399433
4,4,2024-07-23 10:54:54.076000+05:30,3.0,95.192675,96.663373,447.379130,218.706219,1205.0,28.418231,2.0,26.207293,6.0,4.117083,8.0,3.850000,5.0,88.281769,95.129170,13.000000,0.600000,62.0,4144.894647,37.772926,0.0,0.0,0.0,0.0,346.717781,290.112888,0.0,337.038613,216.642832,0,255.746391


In [4]:
from pycaret.classification import *
from imblearn.over_sampling import RandomOverSampler


ignore = ['Unnamed: 0','Anomaly_Score','timestamps']

# Initialize the setup
clf1 = setup(data=df,target='Anomaly',
             session_id=123,
             train_size=0.7,
             preprocess=True,
             imputation_type='simple',
             categorical_imputation='mode',
             numeric_imputation='mean',
             normalize=True,
             ignore_features=ignore,
             normalize_method='robust',
             combine_rare_levels=True,
             rare_level_threshold=0.1,
             data_split_stratify=True,
             fold_strategy='stratifiedkfold',
             fold=3,
             fold_shuffle=True,
             n_jobs=None,
             silent=True,fix_imbalance=True,
             fix_imbalance_method=RandomOverSampler(),
             ignore_low_variance=True,
             handle_unknown_categorical=True,
             unknown_categorical_method='least_frequent',
             feature_selection=True,
             feature_selection_method= "classic",
            #  log_experiment=True,
            #  experiment_name="AnomalyDetection using Pycaret"
             )

,Description,Value
0,session_id,123
1,Target,Anomaly
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(17280, 34)"
5,Missing Values,True
6,Numeric Features,20
7,Categorical Features,10
8,Ordinal Features,False
9,High Cardinality Features,False


In [5]:
import mlflow
mlflow.set_tracking_uri("http://localhost:5000")

In [6]:
# Comparing models to find the best one
best = compare_models(sort = 'recall',n_select=3)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
ada,Ada Boost Classifier,0.9487,0.9746,0.8578,0.4935,0.6261,0.6007,0.6275,2.0233
gbc,Gradient Boosting Classifier,0.9567,0.9721,0.7884,0.5488,0.6463,0.6241,0.6362,8.6300
qda,Quadratic Discriminant Analysis,0.2906,0.5013,0.7354,0.0527,0.0977,0.0050,-0.0010,0.1000
svm,SVM - Linear Kernel,0.8074,0.0000,0.6611,0.1589,0.2561,0.1908,0.2558,0.2600
lr,Logistic Regression,0.7914,0.7952,0.6578,0.1466,0.2398,0.1721,0.2384,0.2200
ridge,Ridge Classifier,0.8035,0.0000,0.6462,0.1532,0.2476,0.1814,0.2447,0.0500
lda,Linear Discriminant Analysis,0.8035,0.7971,0.6462,0.1532,0.2476,0.1814,0.2447,0.1800
lightgbm,Light Gradient Boosting Machine,0.9756,0.9841,0.6115,0.8616,0.7146,0.7023,0.7139,0.4933
nb,Naive Bayes,0.8704,0.8287,0.5883,0.2125,0.3122,0.2576,0.2988,0.0533
knn,K Neighbors Classifier,0.9521,0.7550,0.5189,0.5210,0.5198,0.4947,0.4947,6.3633


In [7]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [8]:
predict_model(best)

AttributeError: 'list' object has no attribute 'predict'

In [ ]:
# Save the model
save_model(best, 'trainedModel')

In [ ]:
loaded_model = load_model('trainedModel')

In [ ]:
!mlflow ui